In [7]:
from clickhouse_driver import Client
import pandas
import geopandas
import os

In [8]:
CLICKHOUSE_HOST=os.getenv("CLICKHOUSE_HOST")
CLICKHOUSE_USERNAME=os.getenv("CLICKHOUSE_USERNAME")
CLICKHOUSE_PASSWORD=os.getenv("CLICKHOUSE_PASSWORD")
client = Client(CLICKHOUSE_HOST, user=CLICKHOUSE_USERNAME, password=CLICKHOUSE_PASSWORD)

# Aggregate: Last record per weather station

In [3]:
client.execute("""
CREATE MATERIALIZED VIEW IF NOT EXISTS weatherStationObservation_last_shard on CLUSTER `cluster`
ENGINE = ReplicatedAggregatingMergeTree(
    '/clickhouse/tables/{shard}/weatherStationObservation_last_shard',
    '{replica}' )
ORDER BY `station_id`
AS SELECT
    `station_id`,
       maxState(`measurement_moment`) as measurement_moment_last,
    argMaxState(`temperature_ambient`, `measurement_moment`) as temperature_ambient_last,
    argMaxState(`temperature_ground`, `measurement_moment`) as temperature_ground_last,
    argMaxState(`humidity`, `measurement_moment`) as humidity_last,
    argMaxState(`pressure`, `measurement_moment`) as pressure_last,
    argMaxState(`wind_speed`, `measurement_moment`) as wind_speed_last,
    argMaxState(`precipitation`, `measurement_moment`) as precipitation_last,
    argMaxState(`irradiance`, `measurement_moment`) as irradiance_last
FROM weatherStationObservation_shard
GROUP BY station_id
""")

client.execute("""
CREATE TABLE IF NOT EXISTS `weatherStationObservation_last_dist` ON CLUSTER `cluster` 
AS weatherStationObservation_last_shard
ENGINE = Distributed(cluster, default, weatherStationObservation_last_shard, station_id)
""")

client.execute("""CREATE OR REPLACE VIEW weatherStationObservation_last ON CLUSTER `cluster` as 
SELECT 
    station_id,
       maxMerge(`measurement_moment_last`) as measurement_moment,
    argMaxMerge(`temperature_ambient_last`) as temperature_ambient,
    argMaxMerge(`temperature_ground_last`) as temperature_ground,
    argMaxMerge(`humidity_last`) as humidity,
    argMaxMerge(`pressure_last`) as pressure,
    argMaxMerge(`wind_speed_last`) as wind_speed,
    argMaxMerge(`precipitation_last`) as precipitation,
    argMaxMerge(`irradiance_last`) as irradiance
FROM weatherStationObservation_last_dist
GROUP BY station_id""")

[('chi-clickhouse-dataplatform-cluster-0-0', 9000, 0, '', 5, 0),
 ('chi-clickhouse-dataplatform-cluster-1-0', 9000, 0, '', 4, 0),
 ('chi-clickhouse-dataplatform-cluster-1-1', 9000, 0, '', 3, 0),
 ('chi-clickhouse-dataplatform-cluster-2-1', 9000, 0, '', 2, 0),
 ('chi-clickhouse-dataplatform-cluster-2-0', 9000, 0, '', 1, 0),
 ('chi-clickhouse-dataplatform-cluster-0-1', 9000, 0, '', 0, 0)]

In [4]:
data, cols = client.execute("""
SELECT * from weatherStationObservation_last
""", with_column_types=True)
df = pandas.DataFrame(data, columns=[col_name for col_name, _ in cols])
df

,station_id,measurement_moment,temperature_ambient,temperature_ground,humidity,pressure,wind_speed,precipitation,irradiance
0,0,2023-06-06 21:15:48.699842,31.090352,28.928666,0.687358,992.690367,22.775137,12.640128,787.241204
1,77,2023-06-06 21:15:48.720304,29.957717,18.127486,0.597322,1009.599830,12.869841,14.066234,561.337406
2,96,2023-06-06 21:15:48.722204,19.244693,32.670463,0.675088,991.104783,30.438219,19.717185,701.725535
3,10,2023-06-06 21:15:48.701668,24.264873,20.702035,0.766118,991.105837,30.326301,21.013227,656.915155
4,33,2023-06-06 21:15:48.703929,24.238022,25.785636,0.885575,993.241648,26.289770,22.357675,912.271151
...,...,...,...,...,...,...,...,...,...
95,27,2023-06-06 21:15:48.703290,26.192241,27.514049,0.955596,990.815007,11.627501,21.026602,336.509277
96,26,2023-06-06 21:15:48.703214,28.662092,15.048626,0.579652,1002.945626,11.412202,20.068026,767.853773
97,52,2023-06-06 21:15:48.705483,34.438685,11.342880,0.839904,999.438797,33.536352,12.678549,922.685626
98,54,2023-06-06 21:15:48.705688,32.820888,13.416764,0.755108,999.413622,33.967782,12.149255,449.943041


In [5]:
import pandas
import geopandas

data, cols = client.execute("""
SELECT * from weatherStationObservation_last
 JOIN WeatherStation_postgres
 ON WeatherStation_postgres.id = weatherStationObservation_last.station_id
""", with_column_types=True)
df = pandas.DataFrame(data, columns=[col_name for col_name, _ in cols])
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat), crs="EPSG:4326")
gdf['measurement_moment'] = df['measurement_moment'].dt.strftime('%Y-%m-%d %H:%M:%S')
gdf.explore("station_name", legend=False)

# Aggregate: Aggregate / day

In [95]:
client.execute("""
CREATE MATERIALIZED VIEW IF NOT EXISTS weatherStationObservation_day_shard on CLUSTER `cluster`
ENGINE = ReplicatedAggregatingMergeTree(
    '/clickhouse/tables/{shard}/weatherStationObservation_day_shard',
    '{replica}' )
ORDER BY (`station_id`, `day`)
AS SELECT
    `station_id`,
    toDate(toStartOfDay(`measurement_moment`)) AS `day`,
    
    countState(`measurement_moment`) as `count`,
    
    avgState(`temperature_ambient`) as temperature_ambient_avg,
    quantilesState(0.1, 0.9)(`temperature_ambient`) as temperature_ambient_quantiles,
    stddevSampState(`temperature_ambient`) as temperature_ambient_stddev,

    avgState(`temperature_ground`) as temperature_ground_avg,
    quantilesState(0.1, 0.9)(`temperature_ground`) as temperature_ground_quantiles,
    stddevSampState(`temperature_ground`) as temperature_ground_stddev,

    avgState(`humidity`) as humidity_avg,
    quantilesState(0.1, 0.9)(`humidity`) as humidity_quantiles,
    stddevSampState(`humidity`) as humidity_stddev,

    avgState(`pressure`) as pressure_avg,
    quantilesState(0.1, 0.9)(`pressure`) as pressure_quantiles,
    stddevSampState(`pressure`) as pressure_stddev,

    avgState(`wind_speed`) as wind_speed_avg,
    quantilesState(0.1, 0.9)(`wind_speed`) as wind_speed_quantiles,
    stddevSampState(`wind_speed`) as wind_speed_stddev,

    avgState(`precipitation`) as precipitation_avg,
    quantilesState(0.1, 0.9)(`precipitation`) as precipitation_quantiles,
    stddevSampState(`precipitation`) as precipitation_stddev,

    avgState(`irradiance`) as irradiance_avg,
    quantilesState(0.1, 0.9)(`irradiance`) as irradiance_quantiles,
    stddevSampState(`irradiance`) as irradiance_stddev

FROM weatherStationObservation_shard
GROUP BY station_id, day
""")

client.execute("""
CREATE TABLE IF NOT EXISTS `weatherStationObservation_day_dist` ON CLUSTER `cluster`
AS weatherStationObservation_day_shard
ENGINE = Distributed(cluster, default, weatherStationObservation_day_shard, station_id)
""")

client.execute("""CREATE OR REPLACE VIEW weatherStationObservation_day ON CLUSTER `cluster` as 
SELECT 
    `station_id`, 
    `day`,
    countMerge(`count`) as `count`,

    avgMerge(`temperature_ambient_avg`) as `temperature_ambient_avg`,
    quantilesMerge(0.1,0.9)(`temperature_ambient_quantiles`) as `temperature_ambient_quantiles`,
    stddevSampMerge(`temperature_ambient_stddev`) as `temperature_ambient_stddev`,
    
    avgMerge(`temperature_ground_avg`) as `temperature_ground_avg`,
    quantilesMerge(0.1,0.9)(`temperature_ground_quantiles`) as `temperature_ground_quantiles`,
    stddevSampMerge(`temperature_ground_stddev`) as `temperature_ground_stddev`,

    avgMerge(`humidity_avg`) as `humidity_avg`,
    quantilesMerge(0.1,0.9)(`humidity_quantiles`) as `humidity_quantiles`,
    stddevSampMerge(`humidity_stddev`) as `humidity_stddev`,
    
    avgMerge(`pressure_avg`) as `pressure_avg`,
    quantilesMerge(0.1,0.9)(`pressure_quantiles`) as `pressure_quantiles`,
    stddevSampMerge(`pressure_stddev`) as `pressure_stddev`,
    
    avgMerge(`wind_speed_avg`) as `wind_speed_avg`,
    quantilesMerge(0.1,0.9)(`wind_speed_quantiles`) as `wind_speed_quantiles`,
    stddevSampMerge(`wind_speed_stddev`) as `wind_speed_stddev`,
    
    avgMerge(`precipitation_avg`) as `precipitation_avg`,
    quantilesMerge(0.1,0.9)(`precipitation_quantiles`) as `precipitation_quantiles`,
    stddevSampMerge(`precipitation_stddev`) as `precipitation_stddev`,
    
    avgMerge(`irradiance_avg`) as `irradiance_avg`,
    quantilesMerge(0.1,0.9)(`irradiance_quantiles`) as `irradiance_quantiles`,
    stddevSampMerge(`irradiance_stddev`) as `irradiance_stddev`

FROM weatherStationObservation_day_dist
GROUP BY `station_id`, `day` """)

[('chi-clickhouse-cluster-0-1', 9000, 0, '', 3, 0),
 ('chi-clickhouse-cluster-0-0', 9000, 0, '', 2, 0),
 ('chi-clickhouse-cluster-1-1', 9000, 0, '', 1, 0),
 ('chi-clickhouse-cluster-1-0', 9000, 0, '', 0, 0)]

In [9]:
data, cols = client.execute("""
SELECT * from weatherStationObservation_day
""", with_column_types=True)
df = pandas.DataFrame(data, columns=[col_name for col_name, _ in cols])

In [12]:
df[df.station_id==1]

,station_id,day,temperature_ambient_avg,temperature_ambient_quantiles_01_05_09,temperature_ground_avg,temperature_ground_quantiles_01_05_09,humidity_avg,humidity_quantiles_01_05_09,pressure_avg,pressure_quantiles_01_05_09,wind_speed_avg,wind_speed_quantiles_01_05_09,precipitation_avg,precipitation_quantiles_01_05_09,irradiance_avg,irradiance_quantiles_01_05_09
80,1,2023-06-07,-7.178822,"[-7.736089507125238, -7.178821787131374, -6.62...",10.647916,"[10.09064781349555, 10.647915533489414, 11.205...",0.850959,"[0.8473544604653337, 0.8509586314183519, 0.854...",994.275069,"[993.9243726682985, 994.2750690880473, 994.625...",15.539901,"[15.354145307320103, 15.539901213984724, 15.72...",13.371248,"[13.299164315315295, 13.371247734375661, 13.44...",656.502100,"[656.4832832569409, 656.502100374718, 656.5209..."
147,1,2023-06-06,-0.339119,"[-15.408298281049433, 0.513413150598983, 12.76...",17.487618,"[2.4184390395713558, 18.340150471219772, 30.59...",0.629927,"[0.39559128363969176, 0.625665774294162, 0.873...",1001.473504,"[985.1256436355833, 1004.7219377887786, 1012.3...",17.819802,"[12.796742382678705, 18.10397952656151, 22.188...",10.548836,"[8.989821295774762, 8.989821295774762, 13.8132...",654.892824,"[652.3782853305715, 655.3940489547299, 656.559..."
